# Sentiment Analysis Of Customer Feedback On Restaurants Using IBM Cloud

In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("zomato.csv")

In [3]:
data_review = dataset['reviews_list']

In [4]:
dataset.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [5]:
dataset.isnull().any()

url                            False
address                        False
name                           False
online_order                   False
book_table                     False
rate                            True
votes                          False
phone                           True
location                        True
rest_type                       True
dish_liked                      True
cuisines                        True
approx_cost(for two people)     True
reviews_list                   False
menu_item                      False
listed_in(type)                False
listed_in(city)                False
dtype: bool

In [6]:
data_review

0        [('Rated 4.0', 'RATED\n  A beautiful place to ...
1        [('Rated 4.0', 'RATED\n  Had been here for din...
2        [('Rated 3.0', "RATED\n  Ambience is not that ...
3        [('Rated 4.0', "RATED\n  Great food and proper...
4        [('Rated 4.0', 'RATED\n  Very good restaurant ...
5        [('Rated 3.0', 'RATED\n  Food 3/5\nAmbience 3/...
6        [('Rated 5.0', 'RATED\n  Awesome food ??Great ...
7        [('Rated 5.0', 'RATED\n  I personally really l...
8        [('Rated 3.0', "RATED\n  I had been to this pl...
9        [('Rated 4.0', "RATED\n  Easy to locate\nVFM 3...
10       [('Rated 4.0', 'RATED\n  We ended up here on a...
11       [('Rated 1.0', "RATED\n \n\nHorrible. Not even...
12       [('Rated 4.0', "RATED\n  Food - 4/5\nAmbience ...
13       [('Rated 2.0', "RATED\n  This is a hookah cafe...
14       [('Rated 3.0', "RATED\n  Ambience is not that ...
15       [('Rated 2.0', 'RATED\n  Not so good place as ...
16       [('Rated 1.0', "RATED\n  This place is right o.

In [7]:
x = []
y = []

In [8]:
x

[]

In [9]:
y

[]

In [10]:
for row_num in range(0,51717):
    # split the revie text at '()
    lst = data_review[row_num].split("('")
    for i in lst:
        if len(i) > 5:
            if i.find("',") != -1:
                single_rev = i.split("',")
                if len(single_rev[0]) > 2:
                    x.append(single_rev[0])
                if len(single_rev[1]) > 2:    
                    y.append(single_rev[1])

In [11]:
import nltk
nltk.download("stopwords") 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BHARGAVI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# to store the final rating
rating_final = []
# to store cleaned revies
review_final = []

In [13]:
import re

In [14]:
for i in range(0,40000):
    data_x = x[i]
    data_x = re.sub('[a-zA-Z]', ' ' , data_x)
    data_x = data_x.split()
    data_x = ''.join(data_x)
    data_x = float(data_x)
    if data_x < 2.5:
        rating_final.append("poor") #poor
    elif data_x >= 2.5 and data_x <= 3.5 :    
        rating_final.append("average") # average
    elif data_x > 3.5:
        rating_final.append("good") #good
        

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
rating_final = le.fit_transform(rating_final)

In [16]:
rating_final = np.array(rating_final)
rating_final = np.expand_dims(rating_final, axis=1)

In [17]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
rates = one.fit_transform(rating_final).toarray()

F:\Users\NITISH\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [18]:
for loop in range(0,40000) : 
    data_y = y[loop]
    data_y = re.sub('[^a-zA-Z]', " ", data_y)
    data_y = data_y.lower()
    data_y = data_y.split()
    data_y = [ps.stem(word) for word in data_y if not word in set(stopwords.words('english'))]
    data_y = ' '.join(data_y)
    review_final.append(data_y)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 20000)
x_final = cv.fit_transform(review_final).toarray()

In [20]:
# saving the vectorizer which would be used as dictionary.
import pickle
pickle.dump(cv, open('cv.pkl','wb'))

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final,rates, test_size = 0.2, random_state = 0)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [23]:
model = Sequential()

In [24]:
model.add(Dense(13264,kernel_initializer = "random_uniform",activation ="relu"))

In [25]:
model.add(Dense(2000,kernel_initializer = "random_uniform",activation ="relu"))

In [26]:
model.add(Dense(2000,kernel_initializer = "random_uniform",activation ="relu"))

In [27]:
model.add(Dense(2000,kernel_initializer = "random_uniform",activation ="relu"))

In [28]:
model.add(Dense(3,kernel_initializer = "random_uniform",activation = "softmax"))

In [29]:
model.compile("adam","categorical_crossentropy",metrics = ["accuracy"])

In [30]:
y_pred = model.predict(x_test)

text =  "The food is okay. average place "
text = re.sub('[^a-zA-Z]', ' ',text)
text = text.lower()
text = text.split()
text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
text = ' '.join(text)

y_p = model.predict(cv.transform([text]))

In [31]:
# saving the model

model.save("zomato_2_analysis.h5")

In [32]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 13264)             175946960 
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              26530000  
_________________________________________________________________
dense_2 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
dense_3 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 6003      
Total params: 210,486,963
Trainable params: 210,486,963
Non-trainable params: 0
_________________________________________________________________
None


In [1]:
model.fit(x_train,y_train)

NameError: name 'model' is not defined

In [ ]:
model.fit(trainDataVecs, y_train, batch_size=64, epochs=11)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
fig = plt.figure()
fig.suptitle('Scatter plot of Actual versus Predicted')
plt.scatter(x=y_pred, y=y_test, marker='.')
plt.xlabel('Predicted')
plt.ylabel('Actual ')
plt.show()